# Calculating State Space Size

The state space is a list of all unique board states reachable during a game of tic-tac-toe. The size of this list is the total number of unique boards.

In [1]:
def print_board(board):
    '''Prints text representation of a board.'''
    values = []
    for cell in board:
        if cell == 1:
            values.append("X")
        elif cell == -1:
            values.append("O")
        else:
            values.append(" ")
    print(f"{values[0]}|{values[1]}|{values[2]}\n-+-+-\n{values[3]}|{values[4]}|{values[5]}\n-+-+-\n{values[6]}|{values[7]}|{values[8]}\n\n")

def rotate_board(board):
    '''Returns the 90 degree rotation of a given board'''
    return tuple(board[i] for i in [6, 3, 0, 7, 4, 1, 8, 5, 2])

def reflect_board(board):
    '''Reflects the given board in the y axis'''
    return tuple(board[i] for i in [2, 1, 0, 5, 4, 3, 8, 7, 6])

def board_transforms(board):
    '''Returns a set containing all possible board transformations'''
    rotations = [board]
    for _ in range(3):
        rotations.append(rotate_board(rotations[-1]))
    reflections = [reflect_board(b) for b in rotations]
    return set(rotations + reflections)

def is_unique(board, seen_boards):
    '''Checks if a provided board has been encountered before'''
    transforms = board_transforms(board)
    for transform in transforms:
        if transform in seen_boards:
            return False
    seen_boards.update(transforms)
    return True

def has_won(board, players):
    '''Detects whether a board contains a winning state'''
    winning_combos = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        [0, 4, 8], [2, 4, 6]
    ]
    for player in players:
        for combo in winning_combos:
            if all(board[i] == player for i in combo):
                return True
    return False

def generate_boards():
    '''Generates all reachable boards in tic-tac-toe'''
    rounds = {0: [tuple([0, 0, 0, 0, 0, 0, 0, 0, 0])]}
    seen_boards = set()
    seen_boards.add(tuple([0, 0, 0, 0, 0, 0, 0, 0, 0]))

    for round in range(1, 10):
        current_boards = []
        for board in rounds[round - 1]:
            if has_won(board, [-1, 1]):
                continue
            player = 1 if round % 2 == 1 else -1
            for i in range(9):
                if board[i] == 0:
                    new_board = tuple(board[j] if j != i else player for j in range(9))
                    if is_unique(new_board, seen_boards):
                        current_boards.append(new_board)
        rounds[round] = current_boards
    return rounds, seen_boards

We can calculate the number of unique board states after each move. The initial 1 state, 3 states and 12 states found line up with the illustrations seen of the first three layers of the game tree.

In [2]:
# Finds state space size at each layer
rounds, seen_boards = generate_boards()

total = 0
for i in range(0, 10):
    total += len(rounds[i])
    print(f"Move {i}: {len(rounds[i])} state{'' if i == 0 else 's'}")
print(f"Total of {total} board states")

Move 0: 1 state
Move 1: 3 states
Move 2: 12 states
Move 3: 38 states
Move 4: 108 states
Move 5: 174 states
Move 6: 204 states
Move 7: 153 states
Move 8: 57 states
Move 9: 15 states
Total of 765 board states


We can easily calculate the number of wins for X, wins for O, drawn games, and mid game states. There are 138 unique terminal states.

In [3]:
# Finds total number of wins for each player
x_wins = []
o_wins = []
draws = []
unfinished = []

for i in range(0, 10):
    for board in rounds[i]:
        if has_won(board, [1]):
            x_wins.append(board)
        elif has_won(board, [-1]):
            o_wins.append(board)
        elif all(cell != 0 for cell in board):
            draws.append(board)
        else:
            unfinished.append(board)

print(f"X Wins: {len(x_wins)}\nO Wins: {len(o_wins)}\nDraws: {len(draws)}\nNot Finished: {len(unfinished)}")

X Wins: 91
O Wins: 44
Draws: 3
Not Finished: 627


We can check how many intermediate board states are present after each move by removing the winning/drawing states from the sum. Note that the even moves have counts 1 + 12 + 108 + 183 = 304, the number of matchboxes needed for MENACE. The 34 at move 8 are ignored as there is only 1 move left for MENACE to play, so it is forced to take the move.

In [4]:
for i in range(0, 10):
    total = 0
    for board in rounds[i]:
        if board in unfinished:
            total += 1
    print(f"Incomplete boards at Move {i}: {total}")

print(f"\nTotal MENACE matchboxes needed: {1 + 12 + 108 + 183}")

Incomplete boards at Move 0: 1
Incomplete boards at Move 1: 3
Incomplete boards at Move 2: 12
Incomplete boards at Move 3: 38
Incomplete boards at Move 4: 108
Incomplete boards at Move 5: 153
Incomplete boards at Move 6: 183
Incomplete boards at Move 7: 95
Incomplete boards at Move 8: 34
Incomplete boards at Move 9: 0

Total MENACE matchboxes needed: 304


We can use the `print_board` function to display specific board states in plaintext.

In [5]:
# Prints all drawn games
for board in draws:
    print_board(board)

X|O|X
-+-+-
O|X|X
-+-+-
O|X|O


X|O|X
-+-+-
O|O|X
-+-+-
X|X|O


X|O|X
-+-+-
X|O|X
-+-+-
O|X|O




# Calculating Game Tree Size

The game tree denotes all possible unique games of tic-tac-toe. The size of this tree represents the total number of unique games. The program sums all routes to the endpoints, which can be accessed from the dictionary at any level. (Note: not particularly efficient, takes a little while to run)

In [6]:
from collections import defaultdict

def forms_board(old_board, new_board, player):
    for i in range(9):
        if old_board[i] == 0:
            trial_board = tuple(old_board[j] if j != i else player for j in range(9))
            if tuple(new_board) in board_transforms(trial_board):
                times_seen[tuple(new_board)] += times_seen[tuple(old_board)]
                return
        
times_seen = defaultdict(int)
times_seen[tuple([0, 0, 0, 0, 0, 0, 0, 0, 0])] = 1

for round in range(1, 10):
    player = 1 if round % 2 == 1 else -1
    for old_board in rounds[round-1]:
        for new_board in rounds[round]:
            forms_board(old_board, new_board, player)


Summing the number of possible routes to the terminal states (wins and draws), we find 26830 unique games.

In [7]:
total = 0
for board in x_wins + o_wins + draws:
    total += times_seen[tuple(board)]
print(total)

26830


It is easy to check the number of possible games leading to each of the 3 end states. Clearly X is at somewhat of an advantage, winning in over 50% of the unique game sequences.

In [8]:
x_win_times = sum(times_seen[tuple(board)] for board in x_wins)
o_win_times = sum(times_seen[tuple(board)] for board in o_wins)
draw_times = sum(times_seen[tuple(board)] for board in draws)

print(f"X wins in {x_win_times} out of {total} unique games")
print(f"O wins in {o_win_times} out of {total} unique games")
print(f"Draws occur in {draw_times} out of {total} unique games")

X wins in 13957 out of 26830 unique games
O wins in 8005 out of 26830 unique games
Draws occur in 4868 out of 26830 unique games
